In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

In [3]:
# Đọc dữ liệu
file_path = 'ner_datasetreference.csv'
dataset = pd.read_csv(file_path, encoding='ISO-8859-1')

# Điền các giá trị NaN trong cột "Sentence #" với giá trị trước đ
dataset['Sentence #'] = dataset['Sentence #'].ffill()

# Tách dữ liệu thành các câu và các nhãn tương ứng
sentences = dataset.groupby("Sentence #")["Word"].apply(list).values
tags = dataset.groupby("Sentence #")["Tag"].apply(list).values


In [4]:
def filter_invalid(sentences, tags):
    filtered_sentences = []
    filtered_tags = []
    for sent, tag in zip(sentences, tags):
        if all(isinstance(word, str) for word in sent):
            filtered_sentences.append(sent)
            filtered_tags.append(tag)
    return filtered_sentences, filtered_tags

sentences, tags = filter_invalid(sentences, tags)

In [6]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences, tags, test_size=0.2, random_state=42
)


In [10]:
def convert_to_spacy_format(sentences, tags):
    nlp = spacy.blank("en")
    db = DocBin()
    for sent, tag in zip(sentences, tags):
        doc = nlp.make_doc(" ".join(sent))
        ents = []
        for i, label in enumerate(tag):
            if label != "O":
                start = len(" ".join(sent[:i])) + (1 if i > 0 else 0)
                end = start + len(sent[i])
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db

In [11]:
train_data_spacy = convert_to_spacy_format(train_sentences, train_tags)
test_data_spacy = convert_to_spacy_format(test_sentences, test_tags)

In [12]:
train_data_spacy.to_disk("train.spacy")
test_data_spacy.to_disk("test.spacy")

In [16]:

config_content = """
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = null

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000

[components]

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM", "PREFIX", "SUFFIX", "SHAPE"]
rows = [5000, 1000, 2500, 2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[initialize]
vectors = ${paths.vectors}
"""
with open("base_config.cfg", "w") as file:
    file.write(config_content)